# Process all file in folder





In [1]:
%load_ext autoreload
%autoreload 2

List the files to process in both conditions wt and sega

In [2]:
import pandas as pd

from pathlib import Path

# list files from the src folder
src = Path('/media/cephfs2/jparham/Joe for Jerome /')
dst = Path('./res')

filelist = pd.read_csv('filelist.csv', index_col=0)
print(f"Number of files {len(filelist)}")
filelist.head()


Number of files 62


,path,name,condition
0,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_0.tif,sega
1,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_12.tif,sega
2,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_13.tif,sega
3,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_3.tif,sega
4,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_11.tif,sega


## Process all file

In [3]:
# from tqdm.notebook import tqdm
# tqdm.pandas()
# import motionquant as mq

# results_path = dst/Path('results.h5') # result
# if results_path.exists():
#     results_path.unlink()

# print(f"Saving results in {results_path}")

# df = []
# for filename in tqdm(filelist['path'].iloc):
#     try:
#         name = filename.stem        
#         img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
#         df.append(mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob))
#         mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
#     except Exception as e:
#         print(f"file '{filename}' could not be processed")
#         print(e)
#         pass
# df = pd.concat(df)


## Parallel processing

In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()
cluster.scale(6)
cluster

LocalCluster(8683ba8f, 'tcp://127.0.0.1:36765', workers=4, threads=12, memory=31.27 GiB)

Create a tag for the result

In [5]:
import git
repo = git.Repo(search_parent_directories=True)
tag = repo.head.object.hexsha[-6:]
print(tag)

8fc314


In [6]:
import motionquant as mq
import dask
from dask.distributed import Lock
import traceback

def process(filename, results_path):
    """Process files and save results in a hf5 file"""
    name = Path(filename).stem        
    img, mask, position, speed, diff, flow, rho, div, blob = mq.process(Path(filename))
    lock = Lock('process-sufo')
    lock.acquire()
    mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
    lock.release()
    df = mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob)
    return df

def process_safe(filename, results_path):
    """Process files catching exceptions"""
    try:
        return process(filename, results_path)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        print(traceback.print_exc())
        pass

results_path = dst/Path(f'results-{tag}.h5') # result
if results_path.exists():
    results_path.unlink()

print(f"Saving results in file '{results_path}'")
tsk = [dask.delayed(process_safe)(filename, results_path) for filename in filelist['path'].iloc]
res = dask.compute(tsk)
df = pd.concat(res[0])

Saving results in file 'res/results-8fc314.h5'
file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS_CM_reg_Cell_2.tif' could not be processed
'str' object has no attribute 'stem'
file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS_CM_Cell_4.tif' could not be processed
'str' object has no attribute 'stem'
NoneNone
file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_11.tif' could not be processed

'str' object has no attribute 'stem'
None
file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20231113_delSegA/CroppedDivisions/02c_delSegA_CM_SyS_end_reg_Cell_5.tif' could not be processed
'str' object has no attribute 'stem'
None
file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20231113_delSegA/CroppedDivisions/02c_delSegA_CM_SyS_start_reg_Cell_6.tif' could not be processed
'str' object has no attribute 'stem'
file 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_saf

file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS_CM_reg_Cell_11.tif' could not be processed
'str' object has no attribute 'stem'
file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_4.tif' could not be processed
'str' object has no attribute 'stem'None

file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_1.tif' could not be processed
'str' object has no attribute 'stem'
None
None
file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS_CM_start_reg_Cell_15.tif' could not be processed
'str' object has no attribute 'stem'
None
file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240306_delSegA/CroppedDivisions/20240306_01a_delSegA_reg_Cell_8.tif' could not be processed
'str' object has no attribute 'stem'
file '/media/cephfs2/jparham/Joe for Jerome /Divi

Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_safe
  File "/tmp/ipykernel_244294/3827943728.py", line 8, in process
AttributeError: 'str' object has no attribute 'stem'
Traceback (most recent call last):
  File "/tmp/ipykernel_244294/3827943728.py", line 20, in process_saf

ValueError: All objects passed were None

In [7]:
df.to_csv(dst/'results.csv')